In [ ]:
import geopandas as gpd

# 1. SHP 파일 불러오기
gdf = gpd.read_file("../data/raw/임상도_서울특별시/11.shp")

# 2. 좌표계 확인 및 변환 (EPSG:5179은 UTM-K 기준)
if gdf.crs != "epsg:5179":
    gdf = gdf.to_crs(epsg=5179)

# 3. 임상코드 기준 필터링
conifer = gdf[gdf["FRTP_CD"] == '1']  # 침엽수
broadleaf = gdf[gdf["FRTP_CD"] == '2']  # 활엽수
mixed = gdf[gdf["FRTP_CD"] == '3']  # 혼효림

# 4. 면적 계산
conifer_area = conifer.geometry.area.sum()
broadleaf_area = broadleaf.geometry.area.sum()
mixed_area = mixed.geometry.area.sum()
total_area = conifer_area + broadleaf_area + mixed_area

# 5. 비율 출력
print(f"총 분석 면적: {total_area/1e6:.2f} km²")
print(f"침엽수림 비율: {conifer_area / total_area * 100:.2f}%")
print(f"활엽수림 비율: {broadleaf_area / total_area * 100:.2f}%")
print(f"혼효림 비율: {mixed_area / total_area * 100:.2f}%")

In [ ]:
# matplotlib 사용하여 시각화
import geopandas as gpd
import matplotlib.pyplot as plt

# plt.rcParams['font.family'] ='AppleGothic'

# SHP 파일 읽기
gdf = gpd.read_file("../data/raw/임상도_서울특별시/11.shp").to_crs(epsg=5179)

# 임상코드 기준 분류 컬럼 추가
def classify_imsang(code):
    if code == '1':
        return '침엽수'
    elif code == '2':
        return '활엽수'
    elif code == '3':
        return '혼효림'
    else:
        return '기타'

gdf["임상분류"] = gdf["FRTP_CD"].apply(classify_imsang)

# 시각화
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(column="임상분류", ax=ax, legend=True, cmap="Set2", edgecolor="black", linewidth=0.1)
plt.title("임상도 침엽수/활엽수 분포 지도")
plt.axis("off")
plt.show()

In [ ]:
# folium 사용하여 시각화
import geopandas as gpd
import folium

# 1. SHP 파일 불러오기 및 좌표계 변환 (4326: 위경도)
gdf = gpd.read_file("../data/raw/임상도_서울특별시/11.shp").to_crs(epsg=4326)

# 2. 임상 코드 → 수종 분류 이름으로 매핑
gdf["수종분류"] = gdf["FRTP_CD"].map({
    '1': '침엽수',
    '2': '활엽수',
    '3': '혼효림'
}).fillna("기타")

# 3. 서울시 중심 좌표
seoul_center = [37.5665, 126.9780]

# 4. 지도 생성
m = folium.Map(location=seoul_center, zoom_start=11, tiles='CartoDB positron')

# 5. 색상 매핑
color_map = {
    '침엽수': 'green',
    '활엽수': 'orange',
    '혼효림': 'purple',
    '기타': 'gray'
}

# 6. 레이어별로 GeoJSON 추가
for label, color in color_map.items():
    layer_data = gdf[gdf["수종분류"] == label]
    if not layer_data.empty:
        folium.GeoJson(
            layer_data,
            name=label,
            style_function=lambda feature, col=color: {
                'fillColor': col,
                'color': 'black',
                'weight': 0.3,
                'fillOpacity': 0.5
            },
            tooltip=folium.GeoJsonTooltip(fields=["수종분류"])
        ).add_to(m)

folium.LayerControl().add_to(m)

# m.save("../output/maps/서울시_임상도_지도.html")

In [ ]:
# Geocoding

import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Point

# CSV 불러오기 (cp949 인코딩 유의)
# 주소 조합
df['주소'] = df['발생장소_시도'].fillna('') + ' ' + \
            df['발생장소_시군구'].fillna('') + ' ' + \
            df['발생장소_읍면'].fillna('') + ' ' + \
            df['발생장소_동리'].fillna('')

# 지오코더 초기화
geolocator = Nominatim(user_agent="fire-mapper")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# tqdm으로 진행 표시
tqdm.pandas()
df['location'] = df['주소'].progress_apply(geocode)

# 위도/경도 컬럼 분리
df['위도'] = df['location'].apply(lambda loc: loc.latitude if loc else None)
df['경도'] = df['location'].apply(lambda loc: loc.longitude if loc else None)

# 유효 좌표만 필터링
df = df.dropna(subset=['위도', '경도'])

# GeoDataFrame 생성
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['경도'], df['위도']))
gdf.set_crs(epsg=4326, inplace=True)

In [ ]:
# 지오코딩 결과 저장
df.to_csv("../data/processed/산불_지오코딩_결과.csv", index=False, encoding='utf-8-sig')
print("✅ 지오코딩 결과가 '산불_지오코딩_결과.csv'에 저장되었습니다.")

In [ ]:
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Point

# 1. 저장된 지오코딩 결과 불러오기
df = pd.read_csv("../data/processed/산불_지오코딩_결과.csv", encoding="utf-8-sig")

# 2. 유효 좌표 필터링
df = df.dropna(subset=['위도', '경도'])

# 3. GeoDataFrame으로 변환 (좌표계 설정은 folium용이므로 4326)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['경도'], df['위도']))
gdf.set_crs(epsg=4326, inplace=True)

# 4. folium 지도 생성 (서울 중심)
m = folium.Map(location=[37.5665, 126.9780], zoom_start=7, tiles="CartoDB positron")

# 5. 산불 발생 위치 마커 추가
for _, row in gdf.iterrows():
    tooltip = f"{row['주소']}<br>피해면적: {row['피해면적_합계']} ha"
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=4,
        color='red',
        fill=True,
        fill_opacity=0.6,
        tooltip=tooltip
    ).add_to(m)

# 6. 지도 저장
# m.save("../output/maps/산불_위치_지도.html")
# print("✅ '산불_위치_지도.html' 파일이 생성되었습니다.")

In [ ]:
import geopandas as gpd
import pandas as pd
import folium

# --- 임상도 불러오기
forest_gdf = gpd.read_file("../data/raw/임상도_서울특별시/11.shp").to_crs(epsg=4326)
forest_gdf["수종분류"] = forest_gdf["FRTP_CD"].map({
    '1': '침엽수',
    '2': '활엽수',
    '3': '혼효림'
}).fillna("기타")

# folium에서 색상 매핑을 위해 color 컬럼 추가
color_dict = {'침엽수': 'green', '활엽수': 'orange', '혼효림': 'purple', '기타': 'gray'}
forest_gdf["color"] = forest_gdf["수종분류"].map(color_dict)

# --- 산불 데이터 불러오기
fire_df = pd.read_csv("../data/processed산불_지오코딩_결과.csv", encoding="utf-8-sig").dropna(subset=["위도", "경도"])
fire_gdf = gpd.GeoDataFrame(fire_df, geometry=gpd.points_from_xy(fire_df["경도"], fire_df["위도"]), crs="EPSG:4326")

# --- 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11, tiles="CartoDB positron")

# --- 임상도 GeoJson 추가 (전체 한 번에 표시)
folium.GeoJson(
    forest_gdf,
    name="임상도",
    style_function=lambda feature: {
        'fillColor': feature['properties']['color'],
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.4
    },
    tooltip=folium.GeoJsonTooltip(fields=["수종분류"])
).add_to(m)

# --- 산불 마커 추가
for _, row in fire_gdf.iterrows():
    tooltip = f"{row['주소']}<br>피해면적: {row['피해면적_합계']} ha"
    folium.CircleMarker(
        location=[row["위도"], row["경도"]],
        radius=4,
        color='red',
        fill=True,
        fill_opacity=0.6,
        tooltip=tooltip
    ).add_to(m)

# --- 레이어 컨트롤 및 저장
folium.LayerControl().add_to(m)
# m.save("산불_임상도_중첩지도.html")
# print("✅ 중첩 지도 생성 완료!")

fire_gdf.head(5)

In [ ]:
# 대한민국 행정지역 데이터
emd = gpd.read_file("../data/raw/emd_20230729/emd.shp")
emd.head()

In [ ]:
from shapely.geometry import box

# 후보 EPSG들
candidate_epsgs = [5174, 5178, 5179, 5181]

for epsg in candidate_epsgs:
    ww = gpd.read_file("../data/raw/emd_20230729/sig.shp")
    ww.crs = f"EPSG:{epsg}"
    ww = ww.to_crs(epsg=4326)
    bounds = ww.total_bounds
    print(f"EPSG:{epsg} → 경계: {bounds}")

In [ ]:
import geopandas as gpd

# 1. SHP 불러오기
ww = gpd.read_file("../data/raw/emd_20230729/emd.shp")

# 2. 좌표계 없음 확인
print("현재 CRS:", ww.crs)  # None일 가능성 큼

# 3. 좌표 범위 보기
print("좌표 경계:", ww.total_bounds)  # [minX, minY, maxX, maxY]

In [ ]:
import geopandas as gpd
import folium

# 1. SHP 파일 불러오기
boundary = gpd.read_file("../data/raw/emd_20230729/sig.shp")

# 2. 좌표계 수동 지정 (서울 TM 중부 좌표계)
boundary.crs = "EPSG:5179" 

# 3. Folium용 EPSG:4326 (위경도)로 변환
boundary = boundary.to_crs(epsg=4326)

# 4. Folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11, tiles="CartoDB positron")

# 5. GeoJson 경계선 추가
folium.GeoJson(
    boundary,
    name="서울시 구 경계선",
    style_function=lambda feature: {
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.01
    }
).add_to(m)

# 6. 저장
folium.LayerControl().add_to(m)
# m.save("../output/map/서울시_구경계_정상.html")
# m

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

#수종별 비율

# 1. 임상도 SHP 파일 로드 (좌표계: EPSG:5179 UTM-K)
forest = gpd.read_file("../data/raw/임상도_서울특별시/11.shp")
if forest.crs != "EPSG:5179":
    forest = forest.to_crs(epsg=5179)

# 2. 수종 이름 추가
forest["수종"] = forest["FRTP_CD"].map({
    '1': '침엽수',
    '2': '활엽수',
    '3': '혼효림'
}).fillna("기타")

# 3. 행정경계 SHP 로드 및 좌표계 통일
emd = gpd.read_file("../data/raw/emd_20230729/emd.shp")
emd.crs = "EPSG:5179"
emd = emd.to_crs(epsg=5179)

# 4. 교차 영역 계산 (클립 대신 overlay 사용)
intersected = gpd.overlay(forest, emd, how="intersection")

# 5. 면적 계산 (m² 단위)
intersected["면적"] = intersected.geometry.area

# 6. 행정동별, 수종별 면적 집계
summary = intersected.groupby(["EMD_ENG_NM", "수종"])["면적"].sum().reset_index()

# 7. 총 면적 기준으로 수종 비율 계산
total_by_emd = summary.groupby("EMD_ENG_NM")["면적"].sum().reset_index(name="총면적")
summary = summary.merge(total_by_emd, on="EMD_ENG_NM")
summary["비율"] = (summary["면적"] / summary["총면적"]) * 100

# 8. 피벗 테이블로 보기 좋게 변환
pivot = summary.pivot_table(index="EMD_ENG_NM", columns="수종", values="비율", fill_value=0).round(2)

In [ ]:
pivot

In [ ]:
import geopandas as gpd
import folium

# 1. 임상도 SHP 파일 로드 및 위경도 좌표계로 변환
forest = gpd.read_file("../data/raw/임상도_서울특별시/11.shp").to_crs(epsg=4326)

# 2. 수종 분류 추가
def classify_imsang(code):
    if code == '1':
        return '침엽수'
    elif code == '2':
        return '활엽수'
    elif code == '3':
        return '혼효림'
    else:
        return '기타'

forest["임상분류"] = forest["FRTP_CD"].apply(classify_imsang)

# 3. 행정동 경계선 SHP 파일 로드 및 좌표계 변환
emd = gpd.read_file("../data/raw/emd_20230729/emd.shp")
emd.crs = "EPSG:5179"  # 실제 좌표계에 맞춰 지정
emd = emd.to_crs(epsg=4326)

# 4. 수종별 색상 매핑
color_map = {
    '침엽수': 'green',
    '활엽수': 'orange',
    '혼효림': 'purple',
    '기타': 'gray'
}

# 5. Folium 지도 생성
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11, tiles='CartoDB positron')

# 6. 수종별 GeoJson 레이어 추가
for label, color in color_map.items():
    layer = forest[forest["임상분류"] == label]
    if not layer.empty:
        folium.GeoJson(
            layer,
            name=label,
            style_function=lambda feature, col=color: {
                'fillColor': col,
                'color': 'black',
                'weight': 0.2,
                'fillOpacity': 0.5
            },
            tooltip=folium.GeoJsonTooltip(fields=["임상분류"])
        ).add_to(m)

# 7. 행정경계 레이어 추가
folium.GeoJson(
    emd,
    name="행정경계",
    style_function=lambda feature: {
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0
    }
).add_to(m)

# 8. 레이어 컨트롤 및 저장
folium.LayerControl().add_to(m)
# m.save("../output/map/서울시_수종_경계_지도.html")